In [1]:
from google import genai

GEMINI_API_KEY="***REMOVED***"
client = genai.Client(api_key=GEMINI_API_KEY)


In [2]:
import os

VIDEO_PATH = 'tesla-real-world-video-q-a/videos'

for video in os.listdir(VIDEO_PATH):
    if video.endswith('.mp4'):
        video_file = client.files.upload(file=VIDEO_PATH + video)
        print(str(video_file) + ' uploaded')


In [3]:
# CACHE THIS SO DONT HAVE TO UPLOAD EVERY TIME
video_names = ['/files/tw665rf6u2em', '/files/adot8mwqx5x8', '/files/9po1avlsm7f9', '/files/yt9a9psrka9r', '/files/p4qf8z7u09gi', '/files/tpen623uy5qr', '/files/cgwmuqbl16zr', '/files/e7hmaqm5i5ha', '/files/hklyvji2hmp7', '/files/iu1dm44rzjxb', '/files/zge9lpjs7h7q', '/files/zb88tzyjuznz', '/files/kwcwvib4tjpv', '/files/o8yib7ohyfxg', '/files/eopu01d6wp0j', '/files/ccske1io7v24', '/files/uxlhl1t9mpxc', '/files/kt7nxnixyfr9', '/files/7255bog4xq89', '/files/2nds4uxi92kd', '/files/esqldu9alume', '/files/ql30433b1vc6', '/files/9ez43wx70qdv', '/files/af34tanig9gc', '/files/pjd5wi63eb1u', '/files/bfwe4wfkjphm', '/files/ve24fszp6dgz', '/files/div8okijpkbh', '/files/splyrhu1g9xs', '/files/jztnog1jagxr', '/files/wqbstailjg60', '/files/dx7pwgwd110u', '/files/sp14h3tvduzy', '/files/n91m9yu0at27', '/files/wz80be455d0s', '/files/302jdw1o6y92', '/files/gknr0u24x64q', '/files/6qtaafyokqd0', '/files/h42kez3dh6lu', '/files/bo774opih334', '/files/f1gugtoepnxh', '/files/4b3rw572v53r', '/files/eglguei4pszy', '/files/1hiy0v2eki7q', '/files/l64min9tuwem', '/files/3wmvt82rh2nw', '/files/scdj8bb5rzc7', '/files/5tcacvfjio6t', '/files/r7f6i7w0ao7g', '/files/z8xnuwwlj74h']


In [ ]:
import pandas as pd

questions = pd.read_csv('tesla-real-world-video-q-a/questions.csv')
questions['answers'] = None
questions.head()

,id,question,answers
0,1,Was ego doing a legal maneuver if its goal is ...,None
1,2,Where can ego legally park on this street? A. ...,None
2,3,What is the best description of the maneuver e...,None
3,4,Why is ego stopped? A. Judah. B. Traffic Light...,None
4,5,"What is the blinker state of the oncoming car,...",None


In [ ]:
from IPython.display import Markdown
import time
from tqdm import trange

def parse_question(question_str):
    Q = question_str.split('A.')[0]
    
    A_str = question_str.split(Q)[1]
    A_str += '&.'
    
    mp = {}
    
    temp = ['A', 'B', 'C', 'D', '&']
    
    for idx in range(len(temp)-1):
        cur_choice = temp[idx]
        nxt_choice = temp[idx+1]
        
        mp[cur_choice] = A_str.split(cur_choice + '.')[1].split(nxt_choice + '.')[0]
    
    return Q, mp

print(parse_question(questions['question'][0]))

('Was ego doing a legal maneuver if its goal is to turn right at the intersection? ', {'A': " It's legal as the lane is empty. ", 'B': " It's illegal as the right turn lane is bloacked by construction. ", 'C': " It's illegal as ego was cutting in other vehicles that were waiting. ", 'D': " It's legal but the lane ahead is way too narrow for ego to pass."})


In [ ]:
def query_model(video_path, prompt):
    # TODO
    video_file = client.files.get(name=video_path)
    result = client.models.generate_content(
                model="gemini-1.5-pro",
                contents=[
                    video_file,
                    prompt
            ]).text
    return result
    

def answer_question(video_file, question, choices):
    
    prompt = question + '\n\nOptions:\n\n'
    
    for choice in choices:
        prompt += '(' + choice + ')' + choices[choice]
    
    return query_model(video_file=video_file)

In [ ]:



for question_number in trange(50):
    if questions.iloc[question_number]['answers'] is not None:
        continue
    video_name = video_names[question_number][1:]
    
    question = questions.iloc[question_number]['question']
    
    
    
    answered = False
    
    while not answered:
        try: 
            

            questions.iloc[question_number]['answers'] = result
            answered = True
            
        except Exception as e:
            print(f"Error on question {question_number}: {e}")
        
        time.sleep(10)

  0%|          | 0/50 [00:00<?, ?it/s]/var/folders/m7/hm_vts8134j67bsp6c2vrwzr0000gn/T/ipykernel_7693/3922890445.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  questions.iloc[question_number]['answers'] = result
  2%|▏         | 1/50 [00:21<17:12, 21.07s/it]


KeyboardInterrupt: 

In [54]:
questions

,id,question,answers
0,1,Was ego doing a legal maneuver if its goal is ...,A
1,2,Where can ego legally park on this street? A. ...,B
2,3,What is the best description of the maneuver e...,C
3,4,Why is ego stopped? A. Judah. B. Traffic Light...,A
4,5,"What is the blinker state of the oncoming car,...",A
5,6,What is the reason ego changed lanes to the le...,B
6,7,What is the correct behavior for ego and why? ...,None
7,8,How many flashing traffic light bulbs are ther...,None
8,9,Which one if more accurate in terms of the dis...,None
9,10,"Ignoring pedestrian lights, how many traffic l...",None
